In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [13]:
#sample documents
documents = [
    "This is my house",
    "Where is hell have you been",
    "Anime is the greatest media ever created"

]



In [14]:
query = "Best media ever"

In [15]:
import re

def preprocess_text(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  return text



In [16]:
preprocessed_doc = [preprocess_text(doc) for doc in documents]

In [17]:
preprocessed_doc

['this is my house',
 'where is hell have you been',
 'anime is the greatest media ever created']

In [18]:
preprocessed_query = preprocess_text(query)

In [19]:
preprocessed_query

'best media ever'

In [20]:
vector = TfidfVectorizer()

In [21]:
X = vector.fit_transform(preprocessed_doc)

In [23]:
X.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.54645401, 0.32274454, 0.        ,
        0.54645401, 0.        , 0.54645401, 0.        , 0.        ],
       [0.        , 0.43238509, 0.        , 0.        , 0.        ,
        0.43238509, 0.43238509, 0.        , 0.2553736 , 0.        ,
        0.        , 0.        , 0.        , 0.43238509, 0.43238509],
       [0.39687454, 0.        , 0.39687454, 0.39687454, 0.39687454,
        0.        , 0.        , 0.        , 0.2344005 , 0.39687454,
        0.        , 0.39687454, 0.        , 0.        , 0.        ]])

In [27]:
query_embedding = vector.transform([preprocessed_query])

In [28]:
query_embedding.toarray()

array([[0.        , 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.70710678,
        0.        , 0.        , 0.        , 0.        , 0.        ]])